In [1]:
import datetime
from haystack.dataclasses import Document

documents = [
    Document(
        content="Decision: Prioritize Project A over Project B for the upcoming quarter. Rationale: Project A has a more immediate impact on client satisfaction. Action Items: Project teams to reallocate resources accordingly.",
        meta={"date": datetime.datetime(2023, 11, 10)},
    ),
    Document(
        content="Decision: Revert back to the original plan, prioritizing Project B. Rationale: Client feedback and market analysis indicate higher long-term potential for Project B. Action Items: Project teams to readjust resources, and communicate changes to stakeholders.",
        meta={"date": datetime.datetime(2023, 11, 12)},
    ),
    Document(
        content="Decision: Allocate 20% of the training budget to online courses. Rationale: Online courses offer cost-effective and flexible learning options. Action Items: HR to update the budget and communicate the changes to employees.",
        meta={"date": datetime.datetime(2023, 11, 11)},
    ),
]

In [2]:
from getpass import getpass

api_key = getpass("OpenAI Key: ")

In [3]:
from haystack import Pipeline
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.generators import OpenAIGenerator

prompt_template = """
You will be provided meeting notes in order. The order is from newest to oldest . Create
a summary of the decisions, indicating the progression.

Meeting notes in order or recency:
{% for document in documents %}
  "Meeting Notes:"
  {{document.content}}
{% endfor %}
"""

prompt_builder = PromptBuilder(template=prompt_template)
llm = OpenAIGenerator(api_key=api_key)

In [4]:
from typing import List, Optional
from haystack import component

@component
class DateRanker():
  def __init__(self, date_field: str = "date"):
    self.date_field = date_field

  @component.output_types(documents=List[Document])
  def run(self, documents: List[Document], date_field: Optional[str] = None):
    if date_field == None:
      date_field = self.date_field

    sorted_documents = sorted(documents, key=lambda x: x.meta[date_field], reverse=True)
    return {"documents": sorted_documents}

In [5]:
date_ranker = DateRanker()

In [6]:
pipe = Pipeline()
pipe.add_component("recentness", date_ranker)
pipe.add_component("prompt_builder", prompt_builder)
pipe.add_component("llm", llm)

pipe.connect("recentness.documents", "prompt_builder.documents")
pipe.connect("prompt_builder", "llm")

In [7]:
result = pipe.run(data={"recentness":{"documents": documents}})
print(result['llm']["replies"][0])

The decisions made during the meetings progressed from prioritizing Project A over Project B for the upcoming quarter, to allocating 20% of the training budget to online courses, and finally reverting back to the original plan of prioritizing Project B. The rationale for the decisions changed based on client feedback, market analysis, and the immediate impact on client satisfaction. The action items included reallocation of resources, updating the budget, and communicating the changes to stakeholders and employees.
